# Delayed starts


In many applications, it is needed to 'delay' the start of particle advection. For example because particles need to be released at different times throughout an experiment. Or because particles need to be released at a constant rate from the same set of locations.

This tutorial will show how this can be done. We start with importing the relevant modules.


In [ ]:
from datetime import timedelta

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

import parcels

First import a `FieldSet` (from the Argo example, in this case)


In [ ]:
# Load the CopernicusMarine data in the Agulhas region from the example_datasets
example_dataset_folder = parcels.download_example_dataset(
    "CopernicusMarine_data_for_Argo_tutorial"
)

ds = xr.open_mfdataset(f"{example_dataset_folder}/*.nc", combine="by_coords")
ds.load()  # load the dataset into memory

fieldset = parcels.FieldSet.from_copernicusmarine(ds)

## Defining initial `time` as particle release


Defining the initial times of particles is done when the `ParticleSet` is defined. Although `time` and `z` are optional arguments (with the earliest time in FieldSet and z=0 as defaults), it is good practice to define them explicitly to ensure expected behavior. The simplest way to delay the start of a particle is to use the `time` argument for each particle.


<div class="alert alert-info">

Note that the `repeatdt` argument used in previous versions of parcels is no longer supported as of v4
</div>

In [ ]:
npart = 10  # number of particles to be released
lon = 32 * np.ones(npart)
lat = np.linspace(-31.5, -30.5, npart, dtype=np.float32)
# release every particle one hour later from the initial fieldset time
time = ds.time.values[0] + np.arange(0, npart) * np.timedelta64(1, "h")
z = np.repeat(ds.depth.values[0], npart)

pset = parcels.ParticleSet(
    fieldset=fieldset, pclass=parcels.Particle, lon=lon, lat=lat, time=time, z=z
)

Then we can execute the `pset` as usual


In [ ]:
output_file = parcels.ParticleFile(
    "DelayParticle_time.zarr", outputdt=np.timedelta64(1, "h")
)

pset.execute(
    parcels.kernels.AdvectionRK4,
    runtime=np.timedelta64(24, "h"),
    dt=np.timedelta64(5, "m"),
    output_file=output_file,
)

And then finally, we can show a movie of the particles. Note that the southern-most particles start to move first.


In [ ]:
%%capture

ds_out_out = xr.open_zarr("DelayParticle_time.zarr", decode_times=False)

fig = plt.figure(figsize=(7, 5), constrained_layout=True)
ax = fig.add_subplot()

ax.set_ylabel("Latitude [deg N]")
ax.set_xlabel("Longitude [deg E]")
ax.set_xlim(31, 33)
ax.set_ylim(-32, -30)

timerange = np.unique(ds_out_out["time"].values[np.isfinite(ds_out_out["time"])])

# Indices of the data where time = 0
time_id = np.where(ds_out_out["time"] == timerange[0])

sc = ax.scatter(ds_out_out["lon"].values[time_id], ds_out_out["lat"].values[time_id])

t = str(timerange[0].astype("timedelta64[h]"))
title = ax.set_title(f"Particles at t = {t}")


def animate(i):
    t = str(timerange[i].astype("timedelta64[h]"))
    title.set_text(f"Particles at t = {t}")

    time_id = np.where(ds_out_out["time"] == timerange[i])
    sc.set_offsets(
        np.c_[ds_out_out["lon"].values[time_id], ds_out_out["lat"].values[time_id]]
    )


anim = FuncAnimation(fig, animate, frames=len(timerange), interval=100)

In [ ]:
HTML(anim.to_jshtml())

## Release particles repeatedly at a set frequency using `np.broadcast_to`


When you want to repeatedly release particles from the same set of locations, you can expand the initial array of particle release locations. Here we show how to release particles at a set frequency `repeatdt`, for a fixed number of releases `nrepeat`. The total number of particles released is then **nrepeat** x **npart**.

In [ ]:
npart = 10  # number of particles to be released

lon_i = 32 * np.ones(npart)
lat_i = np.linspace(-31.5, -30.5, npart, dtype=np.float32)
time_i = np.repeat(ds.time.values[0], npart)
z_i = np.repeat(ds.depth.values[0], npart)

# repeat release at frequency `repeatdt` for `nrepeat` different releases
repeatdt = np.timedelta64(6, "h")
nrepeat = 3

lon = np.broadcast_to(lon_i, (nrepeat, npart))
lat = np.broadcast_to(lat_i, (nrepeat, npart))
time = (
    np.broadcast_to(time_i, (nrepeat, npart))
    + np.arange(0, nrepeat)[:, np.newaxis] * repeatdt
)
z = np.broadcast_to(z_i, (nrepeat, npart))

pset = parcels.ParticleSet(
    fieldset=fieldset, pclass=parcels.Particle, lon=lon, lat=lat, time=time, z=z
)

Now we again define an output file and execute the `pset` as usual.


In [ ]:
output_file = parcels.ParticleFile(
    "DelayParticle_releasedt.zarr", outputdt=timedelta(hours=1)
)

pset.execute(
    parcels.kernels.AdvectionRK4,
    runtime=timedelta(hours=24),
    dt=timedelta(minutes=5),
    output_file=output_file,
)

And we get an animation where a new particle is released every 6 hours from each start location


In [ ]:
%%capture

ds_out = xr.open_zarr("DelayParticle_releasedt.zarr", decode_times=False)

fig = plt.figure(figsize=(7, 5), constrained_layout=True)
ax = fig.add_subplot()

ax.set_ylabel("Latitude [deg N]")
ax.set_xlabel("Longitude [deg E]")
ax.set_xlim(31, 33)
ax.set_ylim(-32, -30)

timerange = np.unique(ds_out["time"].values[np.isfinite(ds_out["time"])])

# Indices of the data where time = 0
time_id = np.where(ds_out["time"] == timerange[0])

sc = ax.scatter(ds_out["lon"].values[time_id], ds_out["lat"].values[time_id])

t = str(timerange[0].astype("timedelta64[h]"))
title = ax.set_title(f"Particles at t = {t}")


def animate(i):
    t = str(timerange[i].astype("timedelta64[h]"))
    title.set_text(f"Particles at t = {t}")

    time_id = np.where(ds_out["time"] == timerange[i])
    sc.set_offsets(np.c_[ds_out["lon"].values[time_id], ds_out["lat"].values[time_id]])


anim = FuncAnimation(fig, animate, frames=len(timerange), interval=100)

In [ ]:
HTML(anim.to_jshtml())

## Synced `time` in the output file

Note that, because the `outputdt` variable controls the Kernel-loop, all particles are written _at the same time_, even when they start at a non-multiple of `outputdt`.

For example, if your particles start at `time=[0, 1, 2]` and `outputdt=2`, then the times written (for `dt=1` and `endtime=4`) will be


In [ ]:
outtime_expected = np.array(
    [[0, 2, 4], [2, 4, np.datetime64("NaT")], [2, 4, np.datetime64("NaT")]],
    dtype="timedelta64[h]",
)
print(outtime_expected)

In [ ]:
outfilepath = "DelayParticle_nonmatchingtime.zarr"

pset = parcels.ParticleSet(
    fieldset=fieldset,
    pclass=parcels.Particle,
    lat=[-31] * 3,
    lon=[32] * 3,
    time=ds.time.values[0] + np.arange(3) * np.timedelta64(1, "h"),
    z=[0.5] * 3,
)

output_file = parcels.ParticleFile(outfilepath, outputdt=np.timedelta64(2, "h"))
pset.execute(
    parcels.kernels.AdvectionRK4,
    endtime=ds.time.values[0] + np.timedelta64(4, "h"),
    dt=np.timedelta64(1, "h"),
    output_file=output_file,
)

And indeed, the `time` values in the NetCDF output file are as expected


In [ ]:
outtime_infile = xr.open_zarr(outfilepath, decode_times=False).time.values[:]
print(outtime_infile.astype("timedelta64[s]").astype("timedelta64[h]"))

# assert (
#     outtime_expected[np.isfinite(outtime_expected)]
#     == outtime_infile[np.isfinite(outtime_infile)]
# ).all()

Now, for some applications, this behavior may be undesirable; for example when particles need to be analyzed at a same age (instead of at a same time). In that case, we recommend either changing `outputdt` so that it is a common divisor of all start times; or doing multiple Parcels runs with subsets of the original `ParticleSet` (e.g., in the example above, one run with the Particles that start at `time=[0, 2]` and one with the Particle at `time=[1]`). In that case, you will get two files:


In [ ]:
for times in np.array([[0, 2], [1, 3]]).astype("timedelta64[h]"):
    pset = parcels.ParticleSet(
        fieldset=fieldset,
        pclass=parcels.Particle,
        lat=[-31] * len(times),
        lon=[32] * len(times),
        time=ds.time.values[0] + times,
        z=[0.5] * len(times),
    )
    output_file = parcels.ParticleFile(outfilepath, outputdt=np.timedelta64(2, "h"))
    pset.execute(
        parcels.kernels.AdvectionRK4,
        runtime=np.timedelta64(4, "h"),
        dt=np.timedelta64(1, "h"),
        output_file=output_file,
    )
    outtime_infile = xr.open_zarr(outfilepath, decode_times=False).time.values[:]
    print(outtime_infile.astype("timedelta64[s]").astype("timedelta64[h]"))